## Regression - Diabetes [pytorch]

### Data

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
x, y = load_diabetes(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print(x_train_scaled.shape, y_train.shape)
print(x_test_scaled.shape, y_test.shape)

(353, 10) (353,)
(89, 10) (89,)


### [scikit-learn] Modeling and Training

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# sklearn model
model = LinearRegression()
model.fit(x_train_scaled, y_train)

# Training
y_pred = model.predict(x_train_scaled)
acc = r2_score(y_train, y_pred)
print(f'Train Accuracy: {acc:.4f}')

# Evaluation
y_pred = model.predict(x_test_scaled)
acc = r2_score(y_test, y_pred)
print(f'Test  Accuracy: {acc:.4f}')

Train Accuracy: 0.5279
Test  Accuracy: 0.4526


### [pytorch-1] Modeling and Training

- Manual backward propagation
- Manual update of weights and biases

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

def r2_score(y_pred, y_true):
    mean_y = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_y)**2)
    ss_res = torch.sum((y_true - y_pred)**2)  
    return 1 - (ss_res / ss_tot)

## Data
x = torch.tensor(x_train_scaled).float()
y = torch.tensor(y_train).float().view(-1, 1)
print(x.shape, y.shape)

torch.Size([353, 10]) torch.Size([353, 1])


In [4]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

w1 = torch.randn(input_size, hidden_size)
b1 = torch.zeros(hidden_size)
w2 = torch.randn(hidden_size, output_size)
b2 = torch.zeros(output_size)

## Train
n_epochs = 10000
learning_rate = 0.1

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = z2

    loss = torch.mean((out - y)**2)
    score = r2_score(out, y)

    # # Backward progapation
    grad_out = 2 * (out - y) / y.shape[0]
    grad_z2 = grad_out
    grad_w2 = torch.mm(a1.T, grad_z2)
    grad_b2 = torch.sum(grad_out, dim=0)

    grad_a1 = torch.mm(grad_z2, w2.T)
    grad_z1 = a1 * (1 - a1) * grad_a1
    grad_w1 = torch.mm(x.T, grad_z1)
    grad_b1 = torch.sum(grad_z1, dim=0)

    # # Update weights and biases
    w1 -= learning_rate * grad_w1
    b1 -= learning_rate * grad_b1
    w2 -= learning_rate * grad_w2
    b2 -= learning_rate * grad_b2

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 2093.73 score: 0.6554
[2000/10000] loss: 1979.78 score: 0.6742
[3000/10000] loss: 1863.31 score: 0.6934
[4000/10000] loss: 1816.44 score: 0.7011
[5000/10000] loss: 1779.00 score: 0.7072
[6000/10000] loss: 1726.83 score: 0.7158
[7000/10000] loss: 1657.75 score: 0.7272
[8000/10000] loss: 1650.90 score: 0.7283
[9000/10000] loss: 1646.59 score: 0.7290
[10000/10000] loss: 1635.94 score: 0.7308


### [pytorch-2] Modeling and Training

- Automatic backward propagation: torch.autograd.grad()
- Manual update of weights and biases

In [5]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

w1 = torch.randn(input_size, hidden_size).requires_grad_()
b1 = torch.zeros(hidden_size).requires_grad_()
w2 = torch.randn(hidden_size, output_size).requires_grad_()
b2 = torch.zeros(output_size).requires_grad_()

## Train
n_epochs = 10000
learning_rate = 0.1

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = z2

    loss = torch.mean((out - y)**2)
    score = r2_score(out, y)

    # # Backward progapation
    grads = torch.autograd.grad(loss, [w1, b1, w2, b2], create_graph=True)

    # Update weights and biases
    with torch.no_grad():
        w1 -= learning_rate * grads[0]
        b1 -= learning_rate * grads[1]
        w2 -= learning_rate * grads[2]
        b2 -= learning_rate * grads[3]

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 2183.73 score: 0.6406
[2000/10000] loss: 2027.13 score: 0.6664
[3000/10000] loss: 1956.86 score: 0.6780
[4000/10000] loss: 1888.61 score: 0.6892
[5000/10000] loss: 1842.92 score: 0.6967
[6000/10000] loss: 1820.54 score: 0.7004
[7000/10000] loss: 1781.25 score: 0.7069
[8000/10000] loss: 1695.08 score: 0.7210
[9000/10000] loss: 1653.57 score: 0.7279
[10000/10000] loss: 1635.12 score: 0.7309


### [pytorch-3] Modeling and Training

- Automatic backward propagation: loss.backward()
- Manual update of weights and biases

In [6]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

w1 = torch.randn(input_size, hidden_size).requires_grad_()
b1 = torch.zeros(hidden_size).requires_grad_()
w2 = torch.randn(hidden_size, output_size).requires_grad_()
b2 = torch.zeros(output_size).requires_grad_()

## Train
n_epochs = 10000
learning_rate = 0.1

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = z2

    loss = torch.mean((out - y)**2)
    score = r2_score(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        b1 -= learning_rate * b1.grad
        w2 -= learning_rate * w2.grad
        b2 -= learning_rate * b2.grad

        w1.grad.zero_()
        b1.grad.zero_()
        w2.grad.zero_()
        b2.grad.zero_()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 2183.73 score: 0.6406
[2000/10000] loss: 2027.13 score: 0.6664
[3000/10000] loss: 1956.86 score: 0.6780
[4000/10000] loss: 1888.61 score: 0.6892
[5000/10000] loss: 1842.92 score: 0.6967
[6000/10000] loss: 1820.54 score: 0.7004
[7000/10000] loss: 1781.25 score: 0.7069
[8000/10000] loss: 1695.08 score: 0.7210
[9000/10000] loss: 1653.57 score: 0.7279
[10000/10000] loss: 1635.12 score: 0.7309


### [pytorch-4] Modeling and Training

- Automatic backward propagation: loss.backward()
- Automatic update of weights and biases: optimizer

In [7]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

w1 = torch.randn(input_size, hidden_size).requires_grad_()
b1 = torch.zeros(hidden_size).requires_grad_()
w2 = torch.randn(hidden_size, output_size).requires_grad_()
b2 = torch.zeros(output_size).requires_grad_()

## Train
n_epochs = 10000
learning_rate = 0.1

optimizer = optim.SGD([w1, b1, w2, b2], lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = z2

    loss = torch.mean((out - y)**2)
    score = r2_score(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    optimizer.step()
    optimizer.zero_grad()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 2101.86 score: 0.6541
[2000/10000] loss: 2048.99 score: 0.6628
[3000/10000] loss: 1964.38 score: 0.6767
[4000/10000] loss: 1871.65 score: 0.6920
[5000/10000] loss: 1803.78 score: 0.7032
[6000/10000] loss: 1743.77 score: 0.7130
[7000/10000] loss: 1703.55 score: 0.7196
[8000/10000] loss: 1692.61 score: 0.7214
[9000/10000] loss: 1682.86 score: 0.7231
[10000/10000] loss: 1682.41 score: 0.7231


### [pytorch-5] Modeling and Training

- MLP model: torch.nn.Linear()
- Automatic backward propagation: loss.backward()
- Automatic update of weights and biases: optimizer

In [8]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.Sigmoid(),
    nn.Linear(hidden_size, output_size),
)

## Train
n_epochs = 10000
learning_rate = 0.1

loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    out = model(x)

    loss = loss_fn(out, y)
    score = r2_score(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    optimizer.step()
    optimizer.zero_grad()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 3659.79 score: 0.3977
[2000/10000] loss: 3607.84 score: 0.4063
[3000/10000] loss: 3545.18 score: 0.4166
[4000/10000] loss: 3506.80 score: 0.4229
[5000/10000] loss: 3446.41 score: 0.4328
[6000/10000] loss: 3441.26 score: 0.4337
[7000/10000] loss: 3326.53 score: 0.4525
[8000/10000] loss: 3294.39 score: 0.4578
[9000/10000] loss: 3269.97 score: 0.4619
[10000/10000] loss: 3284.90 score: 0.4594


### [pytorch-6] Modeling and Training

- MLP model: torch.nn.Linear() with initialization
- Automatic backward propagation: loss.backward()
- Automatic update of weights and biases: optimizer

In [9]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
        ## initialization
        torch.nn.init.normal_(self.linear1.weight)
        torch.nn.init.normal_(self.linear2.weight)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.sigmoid(x)
        x = self.linear2(x)  
        return x

model = MLP(input_size, hidden_size, output_size)

## Train
n_epochs = 10000
learning_rate = 0.1

loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    out = model(x)

    loss = loss_fn(out, y)
    score = r2_score(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    optimizer.step()
    optimizer.zero_grad()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 2059.90 score: 0.6610
[2000/10000] loss: 1907.40 score: 0.6861
[3000/10000] loss: 1826.25 score: 0.6995
[4000/10000] loss: 1757.20 score: 0.7108
[5000/10000] loss: 1741.95 score: 0.7133
[6000/10000] loss: 1707.32 score: 0.7190
[7000/10000] loss: 1662.56 score: 0.7264
[8000/10000] loss: 1656.79 score: 0.7273
[9000/10000] loss: 1651.06 score: 0.7283
[10000/10000] loss: 1604.59 score: 0.7359
